<a href="https://colab.research.google.com/github/CNIU1997/BA_888/blob/master/BA888_FE_Kiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## upload files and import library
from google.colab import files
#uploaded = files.upload()

import pandas as pd
import numpy as np

 **Import Dataset**

In [0]:
train = pd.read_csv('train_data.csv',index_col=0)
test = pd.read_csv('test_data.csv',index_col=0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,224,225,226) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, comp

In [0]:
X_train = train.iloc[:-2].drop(['X1','PRICE_VAR','year','Class'],axis=1)
X_test = test.iloc[:-2].drop(['X1','PRICE_VAR','year','Class'],axis=1)
#X_train = pd.get_dummies(train, columns=['Sector'])
cols = X_train.select_dtypes(exclude=['float']).columns
cols2 = X_test.select_dtypes(exclude=['float']).columns
X_train[cols] = X_train[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
X_test[cols2] = X_test[cols2].apply(pd.to_numeric, downcast='float', errors='coerce')

y_train = train.iloc[:-2][['Class']]
y_test = test.iloc[:-2][['Class']]

In [0]:
y_train['Class'] =y_train['Class'].apply(pd.to_numeric, downcast='float', errors='coerce')

In [0]:
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [0]:
##check na in y
y_train[pd.isnull(y_train).any(axis=1)]

,Class
7929,NaN
12727,NaN
12728,NaN


In [0]:
y_train.dropna(inplace = True)

In [0]:
X_train.drop(y_train.index[[7929,12727,12728]],inplace=True)



---



**Outliers**

In [0]:
from scipy.stats.mstats import winsorize
## remove outliers 5% higher than the max and 5% lower than the min using for loop and winsorize
for col in X_train: 
     X_train[col] = winsorize(X_train[col], limits=[0.05,0.05])

**Standardization**

In [0]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # calculate mean and standard deviation of train set
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [0]:
X_train_scaled = scaler.transform(X_train) 
X_test_scaled = scaler.transform(X_test) 



---



**Feature Importance**

In [0]:
#check
np.any(np.isnan(y_train))

False

In [0]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


In [0]:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [0]:
feature_imp = pd.Series(clf.feature_importances_,index=X_train.columns).sort_values(ascending=False)
feature_imp


Effect_of_forex_changes_on_cash    0.008434
EPS_Diluted                        0.007469
Revenue_Growth                     0.007293
Gross_Profit_Growth                0.007289
Net_cash_flow_/_Change_in_cash     0.007237
                                     ...   
Preferred_Dividends                0.000938
operatingProfitMargin              0.000133
Sector                             0.000000
cashConversionCycle                0.000000
operatingCycle                     0.000000
Length: 222, dtype: float64

In [0]:
feature_imp[:20]

Effect_of_forex_changes_on_cash           0.008434
EPS_Diluted                               0.007469
Revenue_Growth                            0.007293
Gross_Profit_Growth                       0.007289
Net_cash_flow_/_Change_in_cash            0.007237
EPS                                       0.007113
Financing_Cash_Flow                       0.007111
Weighted_Average_Shares_Growth            0.006984
EPS_Diluted_Growth                        0.006867
Book_Value_per_Share_Growth               0.006801
Operating_Cash_Flow_growth                0.006771
Receivables_growth                        0.006679
Weighted_Average_Shs_Out                  0.006677
EPS_Growth                                0.006592
Weighted_Average_Shares_Diluted_Growth    0.006536
SG&A_Expenses_Growth                      0.006470
Operating_Cash_Flow                       0.006444
Retained_earnings_(deficit)               0.006419
Operating_Income_Growth                   0.006399
Operating_Cash_Flow_per_Share  

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

No handles with labels found to put in legend.


In [0]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6088838268792711
